# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [11]:
#Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
# Data Cleaning:
# drop rows with missing values
spaceship.dropna(inplace=True)
# check if there are any missing values
spaceship.shape

(6606, 14)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
spaceship["Cabin"]=spaceship["Cabin"].str[0]
spaceship.drop(columns=["PassengerId","Name"], inplace=True)
spaceship = pd.get_dummies(spaceship, drop_first=True)
spaceship.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,False,False,True,False,False,False,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,False,False,False,False,False,False,True,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,False,False,False,False,False,False,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,False,False,True,False,False,False,True,False


In [5]:
y = spaceship["Transported"] # 目标变量
# 目标变量是 "Transported"，表示乘客是否被运输。
X = spaceship.drop(columns=["Transported"]) # 特征变量
# 特征变量是除了 "Transported" 之外的所有列。

**Perform Train Test Split**

In [6]:
from sklearn.model_selection import train_test_split
# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# 80% 的数据用于训练，20% 的数据用于测试。
# stratify=y: 保证目标变量在训练集和测试集中分布一致（强烈推荐用于分类问题）

In [9]:
# 对训练集进行标准化操作。
from sklearn.preprocessing import StandardScaler
# 选择需要标准化的数值列.
numeric_features = X_train.select_dtypes(include=[np.number]).columns
# 初始化 scaler，只在训练集上 fit
scaler = StandardScaler()
# 对训练集进行标准化
scaler.fit(X_train[numeric_features])
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
# 用同一个 scaler 去 transform 测试集
X_test[numeric_features] = scaler.transform(X_test[numeric_features])  

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [15]:
# 初始化 Bagging 回归器，使用决策树回归器作为基学习器。
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20), 
                               n_estimators=100, 
                               max_samples=1000)

# 训练 Bagging 回归器
# 使用训练集的数值特征进行训练
bagging_reg.fit(X_train[numeric_features], y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [19]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

pred = bagging_reg.predict(X_test[numeric_features])

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", bagging_reg.score(X_test[numeric_features], y_test))

MAE 0.3167452490497392
RMSE 0.40085991719496766
R2 score 0.3572453071457746


- Random Forests

In [21]:
forest = RandomForestRegressor(n_estimators=100, # 	构建 100 棵决策树
                             max_depth=20) # 每棵树的最大深度，限制模型过拟合
# 训练随机森林回归器
forest.fit(X_train[numeric_features], y_train)

RandomForestRegressor(max_depth=20)

In [22]:
pred = forest.predict(X_test[numeric_features])

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", forest.score(X_test[numeric_features], y_test))

MAE 0.31306342406375554
RMSE 0.4070229645137836
R2 score 0.33732922543364496


- Gradient Boosting

In [24]:
gb_reg = GradientBoostingRegressor(max_depth=3, # 每棵回归树的最大深度，越大越容易过拟合
                                   n_estimators=100) # 总共训练 100 棵树，每棵都是对前一棵残差的拟合
# 训练梯度提升回归器
gb_reg.fit(X_train[numeric_features], y_train)

GradientBoostingRegressor()

In [25]:
pred = gb_reg.predict(X_test[numeric_features])

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test[numeric_features], y_test))

MAE 0.3200660963134034
RMSE 0.4026719059007273
R2 score 0.3514213447931033


- Adaptive Boosting

In [26]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)
# 训练 AdaBoost 回归器
ada_reg.fit(X_train[numeric_features], y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [27]:
pred = ada_reg.predict(X_test[numeric_features])

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test[numeric_features], y_test))

MAE 0.3384464496968722
RMSE 0.4747880308798209
R2 score 0.09830530293304884


Which model is the best and why?

In [33]:
# compare the performance of different models with dictionary and loop.
models = {
    "Bagging Regressor": bagging_reg,
    "Random Forest Regressor": forest,
    "Gradient Boosting Regressor": gb_reg,
    "AdaBoost Regressor": ada_reg
}
results = {}
for name, model in models.items():
    pred = model.predict(X_test[numeric_features])
    results[name] = {
        "MAE": mean_absolute_error(pred, y_test),
        "RMSE": root_mean_squared_error(pred, y_test),
        "R2 score": model.score(X_test[numeric_features], y_test)
    }
# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)

                                  MAE      RMSE  R2 score
Bagging Regressor            0.316745  0.400860  0.357245
Random Forest Regressor      0.313063  0.407023  0.337329
Gradient Boosting Regressor  0.320066  0.402672  0.351421
AdaBoost Regressor           0.338446  0.474788  0.098305
